In [46]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle
rc('xtick', labelsize=14)
rc('ytick', labelsize=14)


rxnames = ["cbrssdr1-honors", "cbrssdr1-bes", "cbrssdr1-fm", "cbrssdr1-browning", "cbrssdr1-meb", "cbrssdr1-ustar",
           "cbrssdr1-hospital", "cnode-wasatch", "cnode-mario", "cnode-moran", "cnode-guesthouse", "cnode-ebc", "cnode-ustar"]
nodes   = len(rxnames)

# INPUTS: coord1, coord2
#    These are (latitude, longitude) coordinates in degrees
#    As given by google maps, for example
# OUTPUT: distance in meters
# Copied from:
#    https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
#
def calcDistLatLong(coord1, coord2):

    R = 6373000.0 # approximate radius of earth in meters

    lat1 = math.radians(coord1[0])
    lon1 = math.radians(coord1[1])
    lat2 = math.radians(coord2[0])
    lon2 = math.radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a    = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    dist = R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return dist


# For each row in the CSV, check and see if it matches one of the radio names.
coord = np.zeros((nodes,2))
with open('powder-deployment.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    #print(reader)
    for row in reader:
        # Find the row number (in rxnames) of the station (node name) in the CSV file
        station = row['Station']
        try: 
            # If you found it, save the (lat,long) coordinate in the 'coord' list
            ind = rxnames.index(station)
            coord[ind,:] = [row['Latitude (deg N)'], row['Longitude (deg E)']]
        except: pass

# Initialize a distance matrix
distance_m = np.zeros((nodes, nodes))
# Loop through the lower triangle of the matrix
for i in range(nodes):
    for j in range(i):
        # Calculate the distance between node i and j. Put it in both i,j and j,i matrix entries.
        distance_m[i,j] = calcDistLatLong(coord[i,:], coord[j,:])
        distance_m[j,i] = distance_m[i,j]

print(np.array_str(distance_m,  precision=1))

[[   0.   857.4 1543.3  932.   892.6  646.4  739.4  907.8  996.2  627.
   351.   305.1  542.9]
 [ 857.4    0.   689.9  562.2  829.4  935.6 1327.9 1114.3 1359.3 1144.5
  1099.   934.8  937.1]
 [1543.3  689.9    0.  1023.6 1345.8 1557.5 1982.3 1677.2 1946.4 1795.6
  1785.9 1617.8 1584.6]
 [ 932.   562.2 1023.6    0.   322.6  593.6 1040.   658.3  932.2  859.3
   991.1  816.1  662.6]
 [ 892.6  829.4 1345.8  322.6    0.   351.   775.2  339.1  614.9  611.3
   833.5  680.6  454.6]
 [ 646.4  935.6 1557.5  593.6  351.     0.   446.4  263.9  435.   267.9
   499.3  372.9  116.3]
 [ 739.4 1327.9 1982.3 1040.   775.2  446.4    0.   508.9  374.6  186.9
   405.6  457.3  397.9]
 [ 907.8 1114.3 1677.2  658.3  339.1  263.9  508.9    0.   276.   397.5
   717.5  621.9  366.1]
 [ 996.2 1359.3 1946.4  932.2  614.9  435.   374.6  276.     0.   377.5
   720.5  691.2  484.8]
 [ 627.  1144.5 1795.6  859.3  611.3  267.9  186.9  397.5  377.5    0.
   346.3  323.9  211.1]
 [ 351.  1099.  1785.9  991.1  833.5  499.

In [51]:
distance_data = (distance_m, rxnames)
# Save the combined data to a file
with open('distance_data.pkl', 'wb') as f:
    pickle.dump(distance_data, f)

header = ', '.join(rxnames)
print(header)
np.savetxt('rooftop_dense_distance_matrix.txt', distance_m, fmt='%.2f', delimiter=', ', newline='\n', header=header)

cbrssdr1-honors, cbrssdr1-bes, cbrssdr1-fm, cbrssdr1-browning, cbrssdr1-meb, cbrssdr1-ustar, cbrssdr1-hospital, cnode-wasatch, cnode-mario, cnode-moran, cnode-guesthouse, cnode-ebc, cnode-ustar
